<a href="https://colab.research.google.com/github/fractalsproject/blobganplay/blob/main/BlobGANDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BlobGAN Playground** 

This is an official Colab notebook for running and interacting with a pre-trained [BlobGAN](https://www.dave.ml/blobgan) checkpoint. We can use this to generate and animate scenes and even to edit real-world images, too. For example, you can move furniture around to remodel a bedroom and enlarge windows to let in some more natural light.

**Make sure you've selected a GPU accelerator (Runtime > Change runtime type > Hardware accelerator > GPU).**

Notebook structure borrowed from Bill Peebles' [GANgealing](https://www.wpeebles.com/gangealing). Thanks Bill!

Run cells by pressing the ▶️ play icon on the left-hand side. Start with the setup cell below!

# Setup

Run this cell to get things started. It may take a few minutes to install all the prerequisites, but you only have to run it once. You may see errors pop up with text like "Your session crashed for an unknown reason", since we restart the environment to update libraries. Please ignore them and move onto the next cell when execution is complete.

In [ ]:
#@title
!git clone --quiet https://github.com/dave-epstein/blobgan.git
import blobgan, os, sys, time
!pip -q install moviepy==1.0.3 tqdm==4.64.0 hydra-core==1.1.2 omegaconf==2.1.2 demoji==1.1.0 mpl_interactions[jupyter]==0.21.0 clean-fid==0.1.23 wandb==0.12.16 lpips==0.1.4 einops==0.4.1 inputimeout==1.0.4 pytorch-lightning==1.6.3 torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
# !pip -q install moviepy==1.0.3 tqdm omegaconf demoji mpl_interactions[jupyter] clean-fid wandb lpips einops inputimeout pytorch-lightning torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!wget -q --show-progress https://github.com/ninja-build/ninja/releases/download/v1.10.2/ninja-linux.zip
!sudo unzip -q ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
from google.colab import output
output.clear()
print('\033[92m' 'Done configuring! Move on to the next cell.', flush=True)
time.sleep(0.2)
os.kill(os.getpid(), 9)

# Select model

You can choose between many different pre-trained models trained on 🛏️ bedrooms, 🍳 kitchens, 🛋️ living rooms, 🍽️ dining rooms, and 🤝🏻 conference rooms.

Downloading and configuring models may take a few minutes. Don't worry about any warnings that get printed out.

To try out a new model, just ▶️🔄 re-run this cell after changing your selection.

In [ ]:
# Set up imports and path
import os, sys
PYTHONPATH = '/env/python:/content/blobgan'
if os.environ['PYTHONPATH'] != PYTHONPATH:
  %matplotlib ipympl
  import matplotlib.pyplot as plt
  import mpl_interactions.ipyplot as iplt
  import numpy as np
  import demoji
  import ipywidgets as widgets
  import torch
  from google.colab import output, files
  from IPython.display import HTML
  from PIL import ImageDraw, Image, ImageFont
  from tqdm.notebook import tqdm
  output.enable_custom_widget_manager()
  sys.path.append('./src')
  os.chdir('blobgan')
  os.environ['PYTHONPATH'] = PYTHONPATH
  import torchvision.transforms.functional as FF
  from utils.colab import *
  from moviepy.editor import *
  from models import BlobGAN
  from torchvision.datasets.utils import download_url
  from utils.io import BED_CONF_COLORS, KLD_COLORS

# Util functions
def viz_score_fn(score):
    score = score.clone()
    score[..., 1:].mul_(2).clamp_(max=1)
    return score
def norm_img(img):
  return img.add(1).div(2).clamp(min=0, max=1)
def for_canvas(img):
  return img[0].round().permute(1,2,0).clamp(min=0,max=255).cpu().numpy().astype(np.uint8)
def draw_labels(img, layout, T):
  font = ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf', 20)
  img = Image.fromarray(img)
  draw = ImageDraw.Draw(img)
  mask = layout['sizes'][0, 1:] > T
  idmap = torch.arange(len(mask))[mask]
  blob = {k: layout[k][0][mask].mul(255).tolist() for k in ('xs','ys')}
  for i, (x,y) in enumerate(zip(blob['xs'], blob['ys'])):
    I = idmap[i]
    _,h = draw.textsize(str(I), font=font)
    w=h
    color = tuple(COLORS[I+1].mul(255).round().int().tolist())
    draw.text((x - w/2,y - h/2), f'{I}', fill=color, stroke_width=1, font=font, stroke_fill=(0,0,0))
  return FF.to_tensor(img).permute(1,2,0), img
def clone_layout(l):
  return {k: (v if isinstance(v, bool) else (v.clone() if torch.is_tensor(v) else {kk: vv.clone().repeat_interleave(1, 0) for kk,vv in v.items()})) for k,v in l.items() if v is not None}

# Configure choice
#@markdown Select your model.
model = '\uD83D\uDECF\uFE0F bedrooms'  #@param ['🛏️ bedrooms', '🍳 kitchens, 🛋️ living rooms, 🍽️ dining rooms', '🤝🏻 conference rooms']
model_name = model

# Download model and set up defaults
ckpt = download_model(model_name)
model = BlobGAN.load_from_checkpoint(ckpt, strict=False).to('cuda')
model.mean_latent = download_mean_latent(model_name).to('cuda')
model.cherry_picked = download_cherrypicked(model_name).to('cuda')
COLORS = KLD_COLORS if 'kitchens' in model_name else BED_CONF_COLORS
noise = [torch.randn((1, 1, 16 * 2**((i + 1)//2), 16 * 2**((i + 1)//2))).to('cuda') for i in range(model.generator_ema.num_layers)]
render_kwargs = {
    'no_jitter': True,
    'ret_layout': True,
    'viz': True,
    'ema': True,
    'viz_colors': COLORS,
    'norm_img': True,
    'viz_score_fn': viz_score_fn,
    'noise': noise
}
size_threshold = -3 if 'bedroom' in model_name else -2
z = None
output.clear()
print('\033[92m' 'Done loading and configuring model!',flush=True)

# Editing scenes

Choose your image, whether ⚙️ generated by our model or (coming soon!) 🌎 from the real world, and get to editing!

**Click and drag anywhere on the image or blob map to move the corresponding object.**

You can also double-click to make objects bigger and right-click to make them smaller, or use the buttons that will appear below after running the cell ▶️.

To try out a new image, just ▶️🔄 re-run the cell after you've made your new selection.

In [ ]:
# Hyperparameters
#@markdown First, choose an image source. For each model, we've pre-selected 10 cherry picked examples that we think are interesting and fun.
image = '\uD83C\uDF52 cherry picked 4\uFE0F\u20E3'  #@param ['🍒 cherry picked 1️⃣', '🍒 cherry picked 2️⃣', '🍒 cherry picked 3️⃣', '🍒 cherry picked 4️⃣', '🍒 cherry picked 5️⃣', '🍒 cherry picked 6️⃣', '🍒 cherry picked 7️⃣', '🍒 cherry picked 8️⃣', '🍒 cherry picked 9️⃣', '🍒 cherry picked 🔟', '🎲 random', '🎨 van gogh (coming soon!)', '📸 your uploaded image (coming soon!)']
#@markdown If you selected `🎲 random`, choose whether to generate a new random image. Check this box to change settings for a random image you already like!
keep_random_image = False #@param {type:"boolean"}
#@markdown If you chose a generated image (`🍒 cherry picked` or `🎲 random`), select the truncation level. We recommend values between 0.3 and 0.5 (on the higher end for non-bedroom data).
truncate = 0.4 #@param {type:"slider", min:0, max:1, step:0.05}
#@markdown We also automatically record interactions with the scene and can render them into a 🎥 video. Set the FPS here.
record_fps = 60 #@param {type:"slider", min:10, max:120, step:10}
#@markdown **Tip:** you can also just click a blob and modify it using the buttons below the image, or using your keyboard ( -/+: shrink/enlarge, arrows: move around, R: reset).

#@markdown When you're all done configuring, run this cell!

# Generate image and blobs
if 'random' in image:
  if z is None or not keep_random_image:
    z = torch.randn((1, 512)).to('cuda')
elif 'cherry' in image:
  cherryidx = [int(v.split()[-1]) - 1 for v in demoji.findall(image.encode('utf-16', 'surrogatepass').decode('utf-16')).values() if 'keycap' in v][0]
  z = model.cherry_picked[cherryidx:cherryidx+1]
else: raise NotImplementedError('Not available yet!')
layout, orig_img = model.gen(z=z, truncate=truncate, **render_kwargs)
orig_blobs = for_canvas(layout['feature_img'].mul(255))
labeled_blobs, labeled_blobs_img = draw_labels(orig_blobs, layout, size_threshold)

blobs = DraggableBlobMap(locals())

# Coming soon: upload your own image for editing!

Here's where you will be able to select your own image to try and edit by inverting it into our model. Note that this feature is experimental and will work better on some images than others. 

<!-- You'll begin by **running the cell below and uploading an image**. The cell won't stop running until you upload an image file or cancel. -->

In [ ]:
#@title
from google.colab import files
from pathlib import Path
uploaded = files.upload()
## TODO